## Face Recognition – Unlock Your Computer With Your Face!


### Step 1 - Create Training Data

In [ ]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        # Save file in specified directory with unique name
        file_name_path = './Faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

### Step 2 - Train Model

In [ ]:
cap.release()

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './Faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

model= cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


### Step 3 - Run Our Facial Recognition

In [ ]:
import cv2
import numpy as np
import os
import time

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi



# Open Webcam
cap = cv2.VideoCapture(0)

while True:
    #Capturing image
    ret, frame = cap.read()
    
    #Calling function for face detection
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (240, 180), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        #confidence>=85 : detect my face and send messages to different social media platforms
        if confidence >=85:
            cv2.putText(image, "Hey Aditya", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        
            cv2.imshow('Face Recognition', image )
           
            send_mail(image)
            print("Image mailed to receiver sucessfully..")
            
            print("\n Enter whatsapp number of recipient ...")
            whatsapp_message()
            print("Whatsapp message send to receiver...")
            
            print("\n Enter the phone no. on which you want to send SMS...")
            send_sms()
            print("SMS sent sucessfully")
            
            telegram_message()
            print("Face detected sucessfully!!")
            break
        
        # confidence<85 : Call function to create ec2 instance and ebs volume and will attach it
        else:
            
            cv2.putText(image, "I dont know who u are", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            
            ec2_ebs_create()
            print(" EC2 instance and volume created and attached sucessfully!!")

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
       
        cv2.imshow('Face Recognition', image )
       
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     

In [ ]:
from email.message import EmailMessage
import smtplib

#send image as attachment to recipient mailid 
def send_mail(image):
    cv2.putText(image, "This is photo of AdityaP", (120, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,255), 2)
    
    #Save image into current directory
    cv2.imwrite('myphoto.jpg',image)
    
    #Opening image in binary format and read it
    with open('myphoto.jpg','rb') as img:
        img_data=img.read()
    
    sender = os.environ.get('MAIL_ADDR')
    password = os.environ.get('MAIL_PASSWD')
    
    #Calling EmailMessage class
    message = EmailMessage()
    message['Subject'] = 'SMTP Mail Request!'
    message['From'] = sender
    message['To'] = 'ksatpute407@gmail.com'
    message.set_content("This is photo of AdityaP.")
    
    #Adding image to be sent as attachment
    message.add_attachment(img_data,maintype='image',subtype='jpg',filename=img.name)
    
    #Connecting SMTP mail server using SSL connection 
    smtpObj = smtplib.SMTP_SSL('smtp.gmail.com',465)
    smtpObj.login(sender,password)
    smtpObj.send_message(message)         
    


In [ ]:
import pywhatkit
import getpass

#Send whatsapp message to my friend
def whatsapp_message():
    PHONE_NO= getpass.getpass()
    pywhatkit.sendwhatmsg_instantly(PHONE_NO, "Hello friend!! how are you doing??")

In [ ]:
#whatsapp_message()

In [ ]:
import boto3
import getpass

#send sms to specified phone number
def send_sms():
    p_no= getpass.getpass()
    client=boto3.client('sns','ap-south-1')
    client.publish(PhoneNumber=p_no,Message="Hello World how are you??")
    

In [ ]:
#send_sms()

In [2]:
import tpvars
import requests

#Sending request to HTTP API URL to send message on telegram channel
def telegram_message():
    URL=tpvars.base_url
    requests.get(URL)


<Response [200]>

In [ ]:

#Create and launch ec2 instance and ebs volume using Terraform
def ec2_ebs_create():
    os.system('terraform init')
    os.system('terraform apply --auto-approve')